# FFF HIPPO notebook 1: merge prep

In [1]:
%load_ext autoreload
%autoreload 2
import hippo
from os import environ, system
from pathlib import Path
import json
from mrich import print
import mrich
import shutil
import molparse as mp

## Get BulkDock HIPPO database

In [2]:
target_name = "TRIM21"

In [3]:
animal = hippo.HIPPO(target_name, Path(environ["BULK"])/"TARGETS"/target_name/f"{target_name}.sqlite")

 Creating HIPPO animal

name = TRIM21

db_path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/TRIM21/TRIM21.sqlite

DEBUG: hippo.Database.__init__()

DEBUG: Database.path = /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/TRIM21/TRIM21.sqlite

DEBUG: hippo.Database.connect()

DEBUG: sqlite3.version='2.6.0'

 Success  Database connected @ /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/TRIM21/TRIM21.sqlite!

 Success  Initialised animal HIPPO("TRIM21")!

## Define merging hypotheses

In [7]:
hits = animal.poses(tag="hits")
hits.add_tag("iter1_hits")

Tagged poses tagged "hits": {P × 83} w/ "iter1_hits"

In [8]:
animal.tags.unique

{'BAD count',
 'GOOD count',
 'MEDIOCRE count',
 'Main status',
 '[Other] upload_1 2025-03-17',
 'hits',
 'iter1_hits'}

In [9]:
merging_hypotheses = {
    # nickname: tag,
    "iter1" : "iter1_hits",
}

json.dump(merging_hypotheses, open("merging_hypotheses.json", "wt"))

In [10]:
# rename things in the README for convenience
system(f"sed -i 's/TARGET_NAME/{target_name}/g' ../README.md")
system(f"sed -i 's/HYPOTHESIS_NICKNAME/{list(merging_hypotheses.keys())[0]}/g' ../README.md")

0

## Knitwork & Fragmenstein inputs

### Create input SDFs & CSVs

In [13]:
for nickname, tag in merging_hypotheses.items():
    
    poses = animal.poses(tag=tag)
    mrich.h3(nickname)
    mrich.var("poses", poses)

    # knitwork
    out_dir = Path(f"../knitwork/{nickname}")
    out_dir.mkdir(parents=True, exist_ok=True)
    poses.to_knitwork(out_dir / f"{nickname}_input.csv", path_root=out_dir, aligned_files_dir="../aligned_files")

    # # fragmenstein
    poses.write_sdf(out_dir / f"{nickname}_hits.sdf")
    Path(f"../fragmenstein/{nickname}").mkdir(parents=True, exist_ok=True)
    shutil.copy(out_dir / f"{nickname}_hits.sdf", Path(f"../fragmenstein/{nickname}") / f"{nickname}_hits.sdf")

    ref_pose = poses[0]
    mrich.var("ref_pose", ref_pose)
    shutil.copy(ref_pose.apo_path, Path(f"../fragmenstein/{nickname}"))

╭───────╮
│ iter1 │
╰───────╯

poses = poses tagged "iter1_hits": {P × 83}

out_path = /opt/xchem-fragalysis-2/maxwin/TRIM21_FFF/knitwork/iter1/iter1_input.csv

path_root = /opt/xchem-fragalysis-2/maxwin/TRIM21_FFF/knitwork/iter1

aligned_files_dir = ../aligned_files

 DISK  Writing /opt/xchem-fragalysis-2/maxwin/TRIM21_FFF/knitwork/iter1/iter1_input.csv...

Output()

 DISK  Writing ../knitwork/iter1/iter1_hits.sdf...

ref_pose = C1->P1: "m0145a"

### Create protonated PDBs for Knitwork

In [14]:
for nickname, tag in merging_hypotheses.items():
    poses = animal.poses(tag=tag)
    for pose in mrich.track(poses):
        path = pose.apo_path
        sys = mp.parse(path,verbosity=False)
        sys.add_hydrogens(pH=7)
        subdir = Path("../aligned_files")/pose.name
        subdir.mkdir(exist_ok=True, parents=True)
        sys.write(subdir / str(path.name).replace('_apo-desolv.pdb', '_apo-desolv-Hs.pdb'), verbosity=1)

Output()

 Warning  fixer.missingAtoms={<Residue 184 (LEU) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LEU)>, <Atom 4 
(CD1) of chain 0 residue 0 (LEU)>, <Atom 5 (CD2) of chain 0 residue 0 (LEU)>]}!

fixer.missingTerminals={<Residue 184 (LEU) of chain 0>: ['OXT']}

 DISK  Writing ../aligned_files/m0145a/m0145a_apo-desolv-Hs.pdb...

 Warning  fixer.missingAtoms={<Residue 184 (LEU) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LEU)>, <Atom 4 
(CD1) of chain 0 residue 0 (LEU)>, <Atom 5 (CD2) of chain 0 residue 0 (LEU)>]}!

...

fixer.missingTerminals={<Residue 184 (LEU) of chain 0>: ['OXT']}

 DISK  Writing ../aligned_files/m0789a/m0789a_apo-desolv-Hs.pdb...

 Warning  fixer.missingAtoms={<Residue 184 (LEU) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LEU)>, <Atom 4 
(CD1) of chain 0 residue 0 (LEU)>, <Atom 5 (CD2) of chain 0 residue 0 (LEU)>]}!

fixer.missingTerminals={<Residue 184 (LEU) of chain 0>: ['OXT']}

 DISK  Writing ../aligned_files/m0790a/m0790a_apo-desolv-Hs.pdb...

## Backup and close database

In [15]:
animal.db.backup()
animal.db.close()

 DISK  Writing /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/TRIM21/TRIM21_0FY62SQ.sqlite...

DEBUG: Copied 207 of 207 pages...

DEBUG: hippo.Database.close()

Closed connection to /opt/xchem-fragalysis-2/maxwin/BulkDock/TARGETS/TRIM21/TRIM21.sqlite
